In [39]:
'''
The results may or may not be reproducible due to dataset shuffling operations,
and also because my sole focus was on creating a visibly beautiful notebook, 
and training ONE good model for submission, for the sake of the assignment requirements.
'''

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import warnings
from tensorflow.python.util import deprecation
from transformers import GPT2Tokenizer, TFGPT2ForSequenceClassification

In [26]:
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow.core.framework.local_rendezvous')
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Loading and Preprocessing Data for Custom Models

The following functions read and preprocess the CSV data on file. The `parse_csv_line()` function works on each line and separates the review and the label while ensuring consistency. The `preprocess()` function also works on each element of the dataset iteratively and vectorizes the review text using an adapted vectorizer. In the `prepare_dataset()` function, we combine both of these functionalities, plus adapt our vectorizer on a subset of the data, and finally return a shuffled, batched, and prefetched dataset along with the adapted vectorizer. Lastly, we save both of these for later use. While the functions look clean and simple, a lot of experimentation went under the hoods which I have cleaned up.

In [36]:
def parse_csv_line(line):
    parts = tf.strings.split(line, sep=',')
    label = tf.strings.to_number(parts[-1], out_type=tf.int32)
    review = tf.strings.reduce_join(parts[:-1], separator=',')
    
    return review, label

# old
def preprocess(text, label, vectorizer):
    # text = tf.expand_dims(text, axis=-1)  # Add an extra dimension for the vectorizer
    text = vectorizer(text)
    return text, label

In [37]:
# Legacy
def prepare_dataset_from_csv(filename, header=True, sample_size=5000, max_tokens=20000, output_sequence_length=500, batch_size=16):
    
    dataset = tf.data.TextLineDataset([filename], num_parallel_reads=tf.data.AUTOTUNE)
        
    # Skip header if present
    if header:
        dataset = dataset.skip(1)
    
    dataset = dataset.map(parse_csv_line, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Create and adapt the TextVectorization layer
    vectorizer = keras.layers.TextVectorization(max_tokens=max_tokens, output_sequence_length=output_sequence_length)
    vectorizer.adapt(dataset.take(sample_size).map(lambda x, y: x))


    # Preprocess text
    dataset = dataset.map(lambda text, label: preprocess(text, label, vectorizer), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset, vectorizer

file_path = 'data/movie.csv'
dataset, vectorizer = prepare_dataset_from_csv(file_path, output_sequence_length=512)

2024-07-27 11:30:46.792163: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 17785007208877762518
2024-07-27 11:30:46.792195: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13876514772406206864
2024-07-27 11:30:46.792201: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2339518255541905773
2024-07-27 11:30:46.792204: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 7353395110688891975
2024-07-27 11:30:46.792206: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 911380512253464011
2024-07-27 11:30:46.792208: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4827334378058113401
2024-07-27 11:30:46.792211: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv it

In [38]:
for i in dataset.take(2).as_numpy_iterator():
    print(i[0].shape, i[1].shape)

(16, 512) (16,)
(16, 512) (16,)


#### Saving the Dataset
I am saving the preprocessed dataset as a TFRecords file.

In [167]:
def _create_feature(value, _type):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
        
    if _type=='label':
        value = [value]   # Because Int64List accepts a list, but label is a scalar
    elif _type=='review':
        value = value.tolist()
        
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _serialize_example(review, label):
    
    feature = {
        'review': _create_feature(review, _type='review'),
        'label': _create_feature(label, _type='label')
    }
    
    example_review = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_review.SerializeToString()

def save_to_tfr(dataset, out_file='data/dataset_prepared.tfrecord'):
    with tf.io.TFRecordWriter(out_file) as writer:
        for batch in dataset:
            reviews, labels = batch
            for review, label in zip(reviews, labels):
                serialized_example = _serialize_example(review.numpy(), label.numpy())
                writer.write(serialized_example)

In [168]:
save_to_tfr(dataset)

2024-07-24 21:31:14.474807: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


The dataset has been saved and can be loaded as follows. Since after storing as TFR, all the in-memory transformations like shuffling and batching are lost, we need to re-apply those.

In [3]:
feature_description = {
    'review': tf.io.FixedLenFeature([500], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

In [4]:
def prepare_dataset_from_tfr(filename, feature_description=feature_description, batch_size=16, shuffle_buffer_size=10000):
    raw_dataset = tf.data.TFRecordDataset([filename])
    
    def _parse_examples(example_review):      # To parse examples
        return tf.io.parse_single_example(example_review, feature_description)
    
    dataset = raw_dataset.map(_parse_examples)
    dataset = dataset.map(lambda example_dict: (example_dict['review'], example_dict['label']))
    dataset = dataset.shuffle(shuffle_buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [6]:
dataset = prepare_dataset_from_tfr('data/dataset_prepared.tfrecord')
for i in dataset.take(1):
    print(i)

(<tf.Tensor: shape=(16, 500), dtype=int64, numpy=
array([[  11,   62,   77, ...,    0,    0,    0],
       [   8,   32,   55, ...,  179,  393,   65],
       [ 102,    3,  552, ...,    0,    0,    0],
       ...,
       [   1,   98,  222, ...,    0,    0,    0],
       [ 212,    9,  648, ..., 2954,  253,   22],
       [  30,    2,  118, ...,    0,    0,    0]])>, <tf.Tensor: shape=(16,), dtype=int64, numpy=array([0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1])>)


2024-07-27 08:30:50.773638: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Model Training
All the preprocessing steps have been done and the dataset is now ready to be trained on. We shall use 4 different models for this purpose:
1. A simple Keras custom DNN with Batch Normalization, Dropout, and other advanced techniques as needed,
2. A simple RNN,
3. An LSTM or GRU, and
4. Transfer Learning using a HuggingFace Transformers with pretrained tokenizers and embeddings.

**1. Custom Deep Neural Network**

In [35]:
BATCHES = 40000 // 16  
TRAIN_SIZE = int(BATCHES * 0.8)
VAL_SIZE = int(BATCHES * 0.1)

train_dataset = dataset.take(TRAIN_SIZE)
val_dataset = dataset.skip(TRAIN_SIZE).take(VAL_SIZE)
test_dataset = dataset.skip(TRAIN_SIZE + VAL_SIZE)


In [21]:
for i in train_dataset.take(1):
    print(i[0].shape, i[1].shape)

(16, 500) (16,)


In [23]:
vocab_size = 20000  
embedding_dim = 128  
input_length = 500  

model_1_dnn = keras.models.Sequential([
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length),
    keras.layers.Flatten(),
    
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),  
    keras.layers.Dropout(0.1),          
    
    keras.layers.Dense(64, activation='relu'),
    keras.layers.BatchNormalization(),  
    keras.layers.Dropout(0.1),          
    
    keras.layers.Dense(1, activation='sigmoid')  
])

model_1_dnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_1_dnn.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=3)])

In [ ]:
loss, accuracy = model_1_dnn.evaluate(test_dataset)

In [34]:
print(f'Loss: {loss}\nAccuracy: {accuracy}')

Loss: 0.32168418169021606
Accuracy: 0.906416654586792


In [37]:
model_1_dnn.save('saved_models/model_1_dnn.keras')

**2. A Simple Recurrent Neural Network**

In [ ]:
model_2_rnn = keras.models.Sequential([
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length),
    
    keras.layers.SimpleRNN(128, return_sequences=True), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.1),
    
    keras.layers.SimpleRNN(64),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.1),
    
    keras.layers.Dense(1, activation='sigmoid')
])

model_2_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Run with caution if using a CPU, took 10+ min on my 4070Ti for some reason
model_2_rnn.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=3)])

In [ ]:
loss_rnn, accuracy_rnn = model_2_rnn.evaluate(test_dataset)

In [38]:
print(f'RNN Loss: {loss_rnn}\nRNN Accuracy: {accuracy_rnn}')

RNN Loss: 0.10939233333333
RNN Accuracy: 0.94426852180481


**3. An RNN with Gated Recurrent Units**

In [ ]:
model_3_gru = keras.models.Sequential(
    [
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length),
    
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.GRU(64),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(1, activation='sigmoid') 
    ]
)

model_3_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
model_3_gru = keras.models.load_model('saved_models/model_3_gru.keras')

In [ ]:
model_3_gru.fit(train_dataset, epochs=4, validation_data=val_dataset, callbacks=[keras.callbacks.EarlyStopping(patience=2)])

In [ ]:
loss_gru, accuracy_gru = model_3_gru.evaluate(test_dataset)

In [44]:
print(f'GRU Loss: {loss_gru}\nGRU Accuracy: {accuracy_gru}')

GRU Loss: 0.02748866192996502
GRU Accuracy: 0.9950000047683716


**Transfer Learning using GPT**

In [ ]:
from transformers import GPT2Tokenizer, TFGPT2ForSequenceClassification

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2ForSequenceClassification.from_pretrained('gpt2')

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})